In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import pandas as pd

import talib as ta

from neo_backtesting import backtest_online, SingleFutureSimulator
from strategy_example.bolling import BollingStrategy, boll_factor

In [3]:
# 回测起始日
START_DATE = '20180301'

# 布林带周期与宽度
N = 100
B = 2

# 回测参数
INIT_CAPITAL = 1e5  # 初始资金，10万
LEVERAGE = 1  # 使用1倍杠杆
FACE_VALUE = 0.01  # 合约面值 0.01
COMM_RATE = 6e-4  # 交易成本万分之 6
LIQUI_RATE = 5e-3  # 爆仓保证金率千分之 3

In [4]:
df = pd.read_feather('../candle_1h.fea')

upper, median, lower = ta.BBANDS(df['close'], timeperiod=N, nbdevup=B, nbdevdn=B, matype=ta.MA_Type.SMA)

df['upper'] = upper
df['median'] = median
df['lower'] = lower

df = df[df['candle_begin_time'] >= START_DATE].reset_index(drop=True)

In [5]:
%%time

simulator = SingleFutureSimulator(
    init_capital=INIT_CAPITAL, 
    face_value=FACE_VALUE, 
    comm_rate=COMM_RATE, 
    liqui_rate=LIQUI_RATE, 
    init_pos=0)

strategy = BollingStrategy(leverage=LEVERAGE, face_value=FACE_VALUE)
backtest_online(
    df, 
    simulator=simulator,
    strategy=strategy,
    factor_columns = ['upper', 'lower', 'median'])

print(df[['candle_begin_time', 'pos', 'equity']].tail().to_markdown(), '\n')

|       | candle_begin_time   |     pos |      equity |
|------:|:--------------------|--------:|------------:|
| 37527 | 2022-06-13 19:00:00 | -321335 | 6.859e+06   |
| 37528 | 2022-06-13 20:00:00 | -321335 | 6.81568e+06 |
| 37529 | 2022-06-13 21:00:00 | -321335 | 6.85607e+06 |
| 37530 | 2022-06-13 22:00:00 | -321335 | 6.97178e+06 |
| 37531 | 2022-06-13 23:00:00 | -321335 | 6.91963e+06 | 

CPU times: user 507 ms, sys: 17.5 ms, total: 525 ms
Wall time: 571 ms
